# US Airline Satisfaction Mini Project 

In this Project, we would like to peform some analysis on a dataset of __[US Airline passenger satisfaction survey](https://www.kaggle.com/datasets/najibmh/us-airline-passenger-satisfaction-survey?resource=download)__.

## Contents
- [Problem](#problem)
- [Data Preparation](#data-preparation)
- [Exploratory Analysis](#exploratory-analysis)
- [Sampling](#sampling)

---

<a id="problem"></a>
## Problem
Based on passenger ratings, we would like to find out how the different indivudal ratings affect the passenger's final decision for a _satisfied_ or _unsatisfied_ with the service provided by US Airline.

**Specifically**:
1. Can we predict if customer would be satified?
1. What are the most important factors that affect customer satisfaction?

---

<a id="data-preparation"></a>
## Data Preparation

### Essential Libraries

Let us begin by importing the essential Python Libraries.

> NumPy : Library for Numeric Computations in Python  
> Pandas : Library for Data Acquisition and Preparation  
> Matplotlib : Low-level library for Data Visualization  
> Seaborn : Higher-level library for Data Visualization  

In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
sb.set() # set the default Seaborn style for graphics

### Import the Dataset
Source: __[US Airline passenger satisfaction survey](https://www.kaggle.com/datasets/najibmh/us-airline-passenger-satisfaction-survey?resource=download)__

Attached file: `satisfaction_v2.csv`  

In [ ]:
satisfactionData = pd.read_csv('satisfaction.csv')
satisfactionData.head()

In [ ]:
satisfactionData.info()

#### Initial Observations
* There are `24` columns and `129880` rows in the dataset.   
* The response variable seems to be `satisfaction_v2`.
* The following `5` columns are non-predictor/unlikly to be predictors: ID, Gender, Customer Type, Age and Type of Travel.
* The remaining `18` columns are potential predictor variables.

#### Predictor Variables
* There are `16` variables identified as `int64` by default. But it seems like only `Flight Distance` and `Departure Delay in Minutes` are actually numeric. The remaining `14` variables are ratings from 0 to 5 and should be considered as Categorical.
* The `Arrivial Delay in Minutes` variable identified as `float64` by default, and it seems to be Numeric.
* The`Class` variable identified as `object` by default, and are most likely Categorical.  
* We noted that `Arrivial Delay in Minutes` seems to be missing some values.

### Dataset Cleaning

<div class="alert alert-block alert-info">
    <b>Missing Values: </b> It's noted that <code>Arrivial Delay in Minutes</code> has count <code>129487</code> instead of <code>129880</code>. This is due to it containing <code>NULL</code> values. We will replace them with <code>0</code> here.
</div>

In [ ]:
# Check count
satisfactionData['Arrival Delay in Minutes'].count()

In [ ]:
satisfactionData['Arrival Delay in Minutes'].fillna(value=0, inplace=True)
# Check count
satisfactionData['Arrival Delay in Minutes'].count()

<div class="alert alert-block alert-info">
    Check that the <code>id</code>s are unique. 
</div>

In [ ]:
len(satisfactionData["id"].unique())

<div class="alert alert-block alert-info">
    <b>Ordinal Categorical Variables</b><br>
    Most ordinal categorical variables are rating types in the <code>int</code> form. No conversion required. <br>
    But we will convert for <code>non-int</code> types <code>Class</code> and <code>Customer Type</code> in 
    <a href="#exploratory-analysis">Exploratory Analysis</a>
</div>

---
<a id="exploratory-analysis"></a>
## Exploratory Analysis

### Response Variable
Lets take a look at the response variable `satisfaction_v2`.

In [ ]:
sb.catplot(y = 'satisfaction_v2', data = satisfactionData, kind = "count")

In [ ]:
countG, countB = satisfactionData['satisfaction_v2'].value_counts()
print("[satisfied] : [neutral/dissatisfied] = [", countG, "] : [", countB, "]")

<div class="alert alert-block alert-info">
    The <code>satisfied</code> to <code>neutral/dissatisfied</code> ratio of <code> 71087 : 58794 </code> is acceptable. We will not perform any rebalancing. 
</div>

### Predictor Variables
Lets take a look at the `18` predictor variables.<br>
We shall split them into the following subcategories.

* Passenger: variables relating to the passenger.
* Service: variables corresponding to the services provided by the airline.
* Others: variables that are do not fall in the above categories.

In [ ]:
satisfactionData.iloc[:,6:24].info()

#### Passenger Variables
Variables relating to the passenger. <br>
**Categorical** : `Class` `Departure/Arrival time convenient` `Type of Travel` `Customer Type` `Gender` <br>
**Numeric** : `Age` <br>

<div class="alert alert-block alert-info">
    <b>Class (Categorical)</b><br>
    The class variable seems to describle the type of flight class the passenger was in.<br>
    Since this is normally choosen by the passenger, we labeled it under <b>Passenger Variables</b><br>
    <b>Values</b><br>
    We observed that there are 3 unique values for <code>Class</code> variable.<br>
    It seems like their ordinal values(ascending) are as follows:<br>
    1: <code>Eco</code> 2: <code>Eco Plus</code> 3: <code>Business</code><br> 
    We will convert them accordingly. <br>
    <b>Distribution</b><br>
    The most common value is <code>Business</code> which is followed closely by <code>Eco</code>.<br>    
    <code>Eco Plus</code> has the least distribution. <br>
    <b>Relation</b><br>
    <code>Business</code> class have the higest satisfied rate while passengers from <code>Eco</code> and
    <code>Eco Plus</code> have higher neutral/disatisfied ratings.
</div>

In [ ]:
print(satisfactionData['Class'].describe())
classTypes = satisfactionData['Class'].unique()
print(classTypes)

In [ ]:
from pandas.api.types import CategoricalDtype
cat_type_class = CategoricalDtype(categories=['Eco', 'Eco Plus', 'Business'], ordered=True)
satisfactionData['Class'] = satisfactionData["Class"].astype(cat_type_class)
satisfactionData['Class'].head()

In [ ]:
sb.catplot(x = 'Class', data = satisfactionData, kind = "count", aspect= 2)

In [ ]:
sb.catplot(x='Class', data = satisfactionData, hue= 'satisfaction_v2', kind = "count", aspect= 2)

In [ ]:
# satisfaction_v2 vs Class
f = plt.figure(figsize=(15, 4))
sb.heatmap(satisfactionData.groupby(['satisfaction_v2', 'Class']).size().unstack(), 
           linewidths = 1, annot = True, fmt = 'g', annot_kws = {"size": 18}, cmap = "BuGn")

<div class="alert alert-block alert-info">
    <b>Departure/Arrival time convenient (Categorical)</b><br>
    This variable seems to describle covenience of the flight departure and arrival times.<br>
    Although flight timings are provided by the airline, the passenger normally pick the timeslot.<br>
    As such, we labeled it under <b>Customer Variables</b><br>
    <b>Values</b><br>
    We observed that there are 6 unique values from 0 to 6.<br>
    It is a <i>rating</i> type variable.<br>
    <b>Distribution</b><br>
    Rating <code>3</code> has the highest distribution followed closely by <code>2</code> and <code>4</code><br>    
    Rating <code>0</code> has the lowest distribution. <br>
    <b>Relation</b>
</div>

In [ ]:
satisfactionData['Departure/Arrival time convenient'].describe()

In [ ]:
sb.catplot(x = 'Departure/Arrival time convenient', data = satisfactionData, kind = "count", aspect= 2)

In [ ]:
sb.catplot(x = 'Departure/Arrival time convenient', data = satisfactionData, 
           hue="satisfaction_v2", kind = "count", aspect= 2)

In [ ]:
# satisfaction_v2 vs Departure/Arrival time convenient
f = plt.figure(figsize=(15, 4))
sb.heatmap(satisfactionData.groupby(['satisfaction_v2', 'Departure/Arrival time convenient']).size().unstack(), 
           linewidths = 1, annot = True, fmt = 'g', annot_kws = {"size": 18}, cmap = "BuGn")

<div class="alert alert-block alert-info">
    <b>Type of Travel (Categorical)</b><br>
    This variable seems to describle type/purpose of travel of the passenger.<br>
    <b>Values</b><br>
    We observed that there are 2 unique values <code>Personal Travel</code> <code>Business travel</code> <br>
    <b>Distribution</b><br>
    <code>Business travel</code> has the higher distribution of 89693. <br>
    <b>Relation</b><br>
    <code>Business travel</code> appears to have higher satisfaction
</div>

In [ ]:
print(satisfactionData['Type of Travel'].describe())
travelTypes = satisfactionData['Type of Travel'].unique()
print(travelTypes)

In [ ]:
sb.catplot(x = 'Type of Travel', data = satisfactionData, kind = "count", 
           aspect= 2, order=['Business travel', 'Personal Travel'] )

In [ ]:
sb.catplot(x = 'Type of Travel', data = satisfactionData, kind = "count", hue="satisfaction_v2",
           aspect= 2, order=['Business travel', 'Personal Travel'])

In [ ]:
# satisfaction_v2 vs Customer Type
f = plt.figure(figsize=(15, 4))
sb.heatmap(satisfactionData.groupby(['satisfaction_v2', 'Type of Travel']).size().unstack(), 
           linewidths = 1, annot = True, fmt = 'g', annot_kws = {"size": 18}, cmap = "BuGn")

<div class="alert alert-block alert-info">
    <b>Customer Type (Categorical)</b><br>
    This variable seems to describle if passenger is a loyal customer.<br>
    <b>Values</b><br>
    We observed that there are 2 unique values <code>Loyal Customer</code> <code>disloyal Customer</code> <br>
    We will assign them the weights as follows: <br>
    1: <code>disloyal Customer</code> 2: <code>Loyal Customer</code> <br>
    <b>Distribution</b><br>
    <code>Loyal Customer</code> has the higher distribution of 106100. <br>
    <b>Relation</b><br>
    <code>Loyal Customer</code> appears to have higher satisfaction rate
</div>

In [ ]:
satisfactionData = pd.read_csv('satisfaction.csv')
print(satisfactionData['Customer Type'].describe())
customerTypes = satisfactionData['Customer Type'].unique()
print(customerTypes)

In [ ]:
cat_type_customer = CategoricalDtype(categories=['disloyal Customer', 'Loyal Customer'], ordered=True)
satisfactionData['Customer Type'] = satisfactionData['Customer Type'].astype(cat_type_customer)
satisfactionData['Customer Type'].head()

In [ ]:
sb.catplot(x = 'Customer Type', data = satisfactionData, kind = "count", aspect= 2)

In [ ]:
sb.catplot(x = 'Customer Type', data = satisfactionData, 
           hue="satisfaction_v2", kind = "count", aspect= 2)

In [ ]:
# satisfaction_v2 vs Customer Type
f = plt.figure(figsize=(15, 4))
sb.heatmap(satisfactionData.groupby(['satisfaction_v2', 'Customer Type']).size().unstack(), 
           linewidths = 1, annot = True, fmt = 'g', annot_kws = {"size": 18}, cmap = "BuGn")

#### Service Variables
_Rating_ type variables that are affected by the service provided by the airline.<br>
<code>Seat comfort</code> 
<code>Food and drink</code> 
<code>Inflight wifi service</code> 
<code>Inflight entertainment</code> 
<code>Online support</code> 
<code>Ease of Online booking</code> 
<code>On-board service</code> 
<code>Leg room service</code> 
<code>Baggage handling</code>
<code>Checkin service</code>
<code>Cleanliness</code>
<code>Online boarding</code>

#### Other Variables
Other variables that are not related to customer or airline service. <br>
<b>Categorical</b>: <code>Gate location </code> <br>
<b>Numeric</b> : 
<code>Flight Distance</code> 
<code>Departure Delay in Minutes</code> 
<code>Arrival Delay in Minutes</code>

<div class="alert alert-block alert-info">
    <b>Gate Location (Categorical)</b><br>
    This variable most likely represent the convenience of the gate location.<br>
    As the airline may not choose their gate location, we did not include under service.<br>
    <b>Values</b><br>
    We observed that there are 6 unique values from 0 to 6.<br>
    It is a <i>rating</i> type variable.<br>
    <b>Distribution</b><br>
    Rating <code>3</code> has the highest distribution. <br>
    Rating <code>0</code> has the lowest distribution. <br>
    <b>Relation</b><br>
    <code>Loyal Customer</code> appears to have higher satisfaction rate
</div>

In [ ]:
satisfactionData['Gate location'].describe()

In [ ]:
f = plt.figure(figsize=(15,8))
sb.catplot(x = 'Gate location', data = satisfactionData, kind = "count", aspect= 2)

In [ ]:
sb.catplot(x = 'Gate location', data = satisfactionData, 
           hue="satisfaction_v2", kind = "count", aspect= 2)

In [ ]:
# satisfaction_v2 vs Gate location
f = plt.figure(figsize=(15, 4))
sb.heatmap(satisfactionData.groupby(['satisfaction_v2', 'Gate location']).size().unstack(), 
           linewidths = 1, annot = True, fmt = 'g', annot_kws = {"size": 18}, cmap = "BuGn")

<div class="alert alert-block alert-info">
    <b>Flight Distance (Numeric)</b><br>
    This variable describes the flight distance most likely in miles.<br>
    <b>Relation</b><br>
    It appears that at below <code>1000</code> miles, satisfaction rate seems to be higher.<br>
</div>

In [ ]:
satisfactionData['Flight Distance'].describe()

In [ ]:
f, axes = plt.subplots(3, 1, figsize=(64, 32))
sb.boxplot(data = satisfactionData['Flight Distance'], orient = "h", ax = axes[0])
sb.histplot(data = satisfactionData['Flight Distance'], ax = axes[1])
sb.violinplot(data = satisfactionData['Flight Distance'], orient = "h", ax = axes[2])

In [ ]:
f = plt.figure(figsize=(15, 8))
sb.boxplot(data = satisfactionData, orient = "h",
          x ='Flight Distance', y = 'satisfaction_v2')

In [ ]:
f = plt.figure(figsize=(15, 8))
sb.kdeplot(data = satisfactionData, x='Flight Distance',hue='satisfaction_v2')

In [ ]:
f = plt.figure(figsize=(15,8))
sb.violinplot(data = satisfactionData, orient = 'h',
              x = 'Flight Distance', y = 'satisfaction_v2')

<div class="alert alert-block alert-info">
    <b>Departure Delay in Minutes (Numeric)</b><br>
    We excluded <code>0</code> departure delays<br>
</div>

In [ ]:
satisfactionData['Departure Delay in Minutes'].describe()

In [ ]:
departDelayData = satisfactionData.loc[~((satisfactionData['Departure Delay in Minutes'] == 0))]
departDelayData['Departure Delay in Minutes'].describe()

In [ ]:
f, axes = plt.subplots(3, 1, figsize=(64, 32))
sb.boxplot(data = departDelayData[['Departure Delay in Minutes']], orient = "h", ax = axes[0])
sb.histplot(data = departDelayData[['Departure Delay in Minutes']], ax = axes[1])
sb.violinplot(data = departDelayData[['Departure Delay in Minutes']], orient = "h", ax = axes[2])
#sb.boxplot(data = departDelayData[['Departure Delay in Minutes']], orient = "h",showfliers=True)

<div class="alert alert-block alert-info">
    <b>Remove outliers</b>
</div>

In [ ]:
departDelayDataClean = departDelayData[['Departure Delay in Minutes','satisfaction_v2']].copy()
# Calculate the quartiles
Q1 = departDelayDataClean.quantile(0.25)
Q3 = departDelayDataClean.quantile(0.75)
# Rule to identify outliers
rule = ((departDelayDataClean < (Q1 - 1.5 * (Q3 - Q1))) 
        | (departDelayDataClean > (Q3 + 1.5 * (Q3 - Q1))))
departDelayOutliers = rule.any(axis = 1)
departDelayOutlierindices = departDelayOutliers.index[departDelayOutliers == True]

# Remove the outliers based on the row indices obtained above
departDelayDataClean.drop(axis = 0,               # 0 drops row 1 drops column
                          index = departDelayOutlierindices, # this takes a list as input
                          inplace = True)         # not overwritten by default 
# Check the clean data
departDelayDataClean['Departure Delay in Minutes'].describe()

In [ ]:
f, axes = plt.subplots(3, 1, figsize=(64, 32))
sb.boxplot(data = departDelayDataClean[['Departure Delay in Minutes']], orient = "h", ax = axes[0])
sb.histplot(data = departDelayDataClean[['Departure Delay in Minutes']], ax = axes[1])
sb.violinplot(data = departDelayDataClean[['Departure Delay in Minutes']], orient = "h", ax = axes[2])

In [ ]:
f = plt.figure(figsize=(15, 8))
sb.boxplot(data = departDelayDataClean, orient = "h",
          x ='Departure Delay in Minutes', y = 'satisfaction_v2')

In [ ]:
f = plt.figure(figsize=(15, 8))
sb.kdeplot(data = departDelayDataClean, x='Departure Delay in Minutes',hue='satisfaction_v2')

In [ ]:
f = plt.figure(figsize=(15,8))
sb.violinplot(data = departDelayDataClean, orient = 'h',
              x = 'Departure Delay in Minutes', y = 'satisfaction_v2')

<div class="alert alert-block alert-info">
    <b>Arrival Delay in Minutes (Numeric)</b><br>
    We excluded <code>0</code> departure delays<br>
</div>

In [ ]:
arriveDelayData = satisfactionData.loc[~((satisfactionData['Arrival Delay in Minutes'] == 0))]
arriveDelayData['Arrival Delay in Minutes'].describe()

In [ ]:
arriveDelayDataClean = arriveDelayData[['Arrival Delay in Minutes','satisfaction_v2']].copy()
# Calculate the quartiles
adQ1 = arriveDelayDataClean.quantile(0.25)
adQ3 = arriveDelayDataClean.quantile(0.75)
# Rule to identify outliers
adrule = ((arriveDelayDataClean < (adQ1 - 1.5 * (adQ3 - adQ1))) 
        | (arriveDelayDataClean > (adQ3 + 1.5 * (adQ3 - adQ1))))
arriveDelayOutliers = adrule.any(axis = 1)
arriveDelayOutlierindices = arriveDelayOutliers.index[arriveDelayOutliers == True]

# Remove the outliers based on the row indices obtained above
arriveDelayDataClean.drop(axis = 0,               # 0 drops row 1 drops column
                          index = arriveDelayOutlierindices, # this takes a list as input
                          inplace = True)         # not overwritten by default 
# Check the clean data
arriveDelayDataClean['Arrival Delay in Minutes'].describe()

In [ ]:
f, axes = plt.subplots(3, 1, figsize=(64, 32))
sb.boxplot(data = arriveDelayDataClean[['Arrival Delay in Minutes']], orient = "h", ax = axes[0])
sb.histplot(data = arriveDelayDataClean[['Arrival Delay in Minutes']], ax = axes[1])
sb.violinplot(data = arriveDelayDataClean[['Arrival Delay in Minutes']], orient = "h", ax = axes[2])

In [ ]:
f, axes = plt.subplots(3, 1, figsize=(64, 32))
sb.boxplot(data = arriveDelayDataClean, orient = "h", ax = axes[0],
          x ='Arrival Delay in Minutes', y = 'satisfaction_v2')
sb.kdeplot(data = arriveDelayDataClean, ax = axes[1],
          x ='Arrival Delay in Minutes', hue = 'satisfaction_v2')
sb.violinplot(data = arriveDelayDataClean, orient = "h", ax = axes[2],
          x ='Arrival Delay in Minutes', y = 'satisfaction_v2')

In [ ]:
test = satisfactionData;
#print(test)
scale_mapper = {0:0, 1:1, 2:2, 3:3, 4:4, 5:5}
scaled = test["Seat comfort"].replace(scale_mapper) 

from pandas.api.types import CategoricalDtype

cat_type_ratings = CategoricalDtype(categories=[0,1,2,3,4,5], ordered=True)
test["Seat comfort"].astype(cat_type_ratings, )
test.info()

In [ ]:
test["Seat comfort"]

In [ ]:
test = pd.DataFrame(satisfactionData)
test['Arrival Delay in Minutes'].fillna(value=0, inplace=True)

In [ ]:
from pandas.api.types import CategoricalDtype

cat_type_ratings = CategoricalDtype(categories=[0,1,2,3,4,5], ordered=True)
cat_type_class = CategoricalDtype(categories=['Eco', 'Eco Plus', 'Business'], ordered=True)

test["Class"] = test["Class"].astype(cat_type_class)
test["Class"]

In [ ]:
test.iloc[:,8:22] = test.iloc[:,8:22].astype(cat_type_ratings)
test.info()

In [ ]:
# Removed redundant date

test = test.drop(['Gender','Customer Type','Age','Type of Travel','Class'], axis = 1)
test.info()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
cats = [0,1,2,3,4,5]
ordi = OrdinalEncoder(categories=[cats])
ordi.fit(test[['Seat comfort']])


In [ ]:
encoded = pd.DataFrame(ordi.transform(test[['Seat comfort']]),columns=['Seat comfort'])
encoded.info()

In [ ]:
test['Seat comfort'].unique()

In [ ]:
y = pd.DataFrame(test['satisfaction_v2'])
X = pd.DataFrame(test.drop('satisfaction_v2', axis = 1))
dectree = DecisionTreeClassifier(max_depth = 5)  # change max_depth to experiment
dectree.fit(X, y)
# Plot the trained Decision Tree
f = plt.figure(figsize=(24,24))
plot_tree(dectree, filled=True, rounded=True, precision=0,
          feature_names=X.columns, 
          class_names=["neutral or dissatisfied","satisfied"])

In [ ]:
# Import essential models and functions from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import plot_tree


# Extract Response and Predictors
y = pd.DataFrame(test['satisfaction_v2'])
X = pd.DataFrame(test.drop('satisfaction_v2', axis = 1))

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

# Decision Tree using Train Data
dectree = DecisionTreeClassifier(max_depth = 3)  # change max_depth to experiment
dectree.fit(X_train, y_train)                    # train the decision tree model

# Plot the trained Decision Tree
f = plt.figure(figsize=(24,24))
plot_tree(dectree, filled=True, rounded=True, 
          feature_names=X_train.columns, 
          class_names=["neutral or dissatisfied","satisfied"])